<a href="https://colab.research.google.com/github/gmoraissc/Projetos_de_Data_Science/blob/main/Sentiment_analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [55]:
# imports
import pandas as pd
import numpy as np
import tweepy, nltk, json
from datetime import datetime, date

In [56]:
access_token="590375619-tkJialGWR4R5ZoXJfWsZlx0DoLUfSDlbUi1hLATZ"
access_token_secret="cGjIJU3YM23f1qxH8LvUKd6Tm05zbOBUI9NcCDxLzJ39T"
bearer_token = 'AAAAAAAAAAAAAAAAAAAAAHeVYAEAAAAA9sjLNUPQJgclF6kKG0lPvLTq9nY%3DwprwxmnxRvmXpbU2CbXAoIGogKKEZCRxKwoi6xNW3au27KlCqk'
consumer_key= 'vS7GEC5Cy7HixHoVKNpCR7cFr'
consumer_secret="0cpjFY2tWCwDoWhqOwr2ZBDy6TbpNOtWnQOpU70V9V0Bwu0JNt"

In [57]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret)

api = tweepy.API(auth)

search_results_popular = api.search(q=['BBB22'],
                            result_type='popular')

search_results_recents = api.search(q=['BBB22'],
                                    result_type='recent')

In [80]:
popular_dict = {}
recents_dict = {}
d1 = date.today().strftime("%d/%m/%Y")

for each_json_tweet in search_results_popular:
    if d1 not in popular_dict.keys():
      tweet = {d1: [json.dumps(each_json_tweet._json)]}
      popular_dict.update(tweet)
   
    else:
      popular_dict[d1].append(json.dumps(each_json_tweet._json))

for each_json_tweet in search_results_recents:
  if d1 not in recents_dict.keys():
    tweet = {d1: [json.dumps(each_json_tweet._json)]}
    recents_dict.update(tweet)
  else:
    recents_dict[d1].append(json.dumps(each_json_tweet._json))

In [81]:
popular_dict[d1][0]

'{"created_at": "Sun Jan 16 02:14:31 +0000 2022", "id": 1482536715915083780, "id_str": "1482536715915083780", "text": "De acordo com apura\\u00e7\\u00e3o da revista Quem, Mar\\u00edlia Mendon\\u00e7a teria se arrependido de gravar com Naiara Azevedo e n\\u00e3o go\\u2026 https://t.co/oUeKkGOwS1", "truncated": true, "entities": {"hashtags": [], "symbols": [], "user_mentions": [], "urls": [{"url": "https://t.co/oUeKkGOwS1", "expanded_url": "https://twitter.com/i/web/status/1482536715915083780", "display_url": "twitter.com/i/web/status/1\\u2026", "indices": [117, 140]}]}, "metadata": {"iso_language_code": "pt", "result_type": "popular"}, "source": "<a href=\\"http://twitter.com/download/iphone\\" rel=\\"nofollow\\">Twitter for iPhone</a>", "in_reply_to_status_id": null, "in_reply_to_status_id_str": null, "in_reply_to_user_id": null, "in_reply_to_user_id_str": null, "in_reply_to_screen_name": null, "user": {"id": 132699857, "id_str": "132699857", "name": "Tracklist #BBB22", "screen_name": 

In [82]:
popular_dict[d1][1]

'{"created_at": "Sat Jan 15 01:28:33 +0000 2022", "id": 1482162756774150146, "id_str": "1482162756774150146", "text": "Saudades do que nunca aconteceu\\u2026 #BBB22 https://t.co/xuKZYNFVme", "truncated": false, "entities": {"hashtags": [{"text": "BBB22", "indices": [33, 39]}], "symbols": [], "user_mentions": [], "urls": [], "media": [{"id": 1482162749413154820, "id_str": "1482162749413154820", "indices": [40, 63], "media_url": "http://pbs.twimg.com/media/FJGzKvIWYAQqLDI.jpg", "media_url_https": "https://pbs.twimg.com/media/FJGzKvIWYAQqLDI.jpg", "url": "https://t.co/xuKZYNFVme", "display_url": "pic.twitter.com/xuKZYNFVme", "expanded_url": "https://twitter.com/tracklist/status/1482162756774150146/photo/1", "type": "photo", "sizes": {"thumb": {"w": 150, "h": 150, "resize": "crop"}, "medium": {"w": 750, "h": 919, "resize": "fit"}, "large": {"w": 750, "h": 919, "resize": "fit"}, "small": {"w": 555, "h": 680, "resize": "fit"}}}]}, "extended_entities": {"media": [{"id": 1482162749413154820, "

In [69]:
popular_list = []

for each_dictionary in popular_dict[d1]:
    tweet_id = each_dictionary['id']
    text = each_dictionary['text']
    favorite_count = each_dictionary['favorite_count']
    retweet_count = each_dictionary['retweet_count']
    created_at = each_dictionary['created_at']
    location = each_dictionary['user']['location']
    followers_count = each_dictionary['user']['followers_count']
    friends_count = each_dictionary['user']['friends_count']
    verified = each_dictionary['user']['verified']
    hashtags = each_dictionary['entities']['hashtags']
    user_mentions = each_dictionary['entities']['user_mentions']
    name = each_dictionary['user']['name']
    screen_name = each_dictionary['user']['screen_name']

    popular_list.append({'tweet_id': str(tweet_id),
                          'text': str(text),
                          'favorite_count': int(favorite_count),
                          'retweet_count': int(retweet_count),
                          'created_at': created_at,
                          'location': location,
                          'followers_count': followers_count,
                          'friends_count': friends_count,
                          'verified': verified,
                          'retweet_count': retweet_count,
                          'favorite_count': favorite_count,
                          'hashtags': hashtags,
                          'user_mentions': user_mentions,
                          'name': name,
                          'screen_name': screen_name,
                          })
    
    tweet_json_popular = pd.DataFrame(my_demo_list, columns = 
                              ['tweet_id', 'text', 
                                'favorite_count', 'retweet_count', 
                                'created_at', 'location',
                                'followers_count', 'friends_count',
                                'verified', 'hashtags',
                                'user_mentions', 'name',
                                'screen_name'])

TypeError: ignored

In [ ]:
recents_list = []

for each_dictionary in popular_dict:
    tweet_id = each_dictionary['id']
    text = each_dictionary['text']
    favorite_count = each_dictionary['favorite_count']
    retweet_count = each_dictionary['retweet_count']
    created_at = each_dictionary['created_at']
    location = each_dictionary['user']['location']
    followers_count = each_dictionary['user']['followers_count']
    friends_count = each_dictionary['user']['friends_count']
    verified = each_dictionary['user']['verified']
    hashtags = each_dictionary['entities']['hashtags']
    user_mentions = each_dictionary['entities']['user_mentions']
    name = each_dictionary['user']['name']
    screen_name = each_dictionary['user']['screen_name']

    popular_list.append({'tweet_id': str(tweet_id),
                          'text': str(text),
                          'favorite_count': int(favorite_count),
                          'retweet_count': int(retweet_count),
                          'created_at': created_at,
                          'location': location,
                          'followers_count': followers_count,
                          'friends_count': friends_count,
                          'verified': verified,
                          'retweet_count': retweet_count,
                          'favorite_count': favorite_count,
                          'hashtags': hashtags,
                          'user_mentions': user_mentions,
                          'name': name,
                          'screen_name': screen_name,
                          })
    
    tweet_json_popular = pd.DataFrame(my_demo_list, columns = 
                              ['tweet_id', 'text', 
                                'favorite_count', 'retweet_count', 
                                'created_at', 'location',
                                'followers_count', 'friends_count',
                                'verified', 'hashtags',
                                'user_mentions', 'name',
                                'screen_name'])

In [ ]:
calendar = {'Jan': 1, 'Feb': 2, 'Mar': 3, 'Apr': 4, 'May': 5, 'Jun': 6, 
                'Jul': 7, 'Aug': 8, 'Sep': 9, 'Oct': 10, 'Nov': 11, 'Dec': 12}
  
tweet_json['Week_day'] = tweet_json['created_at'].str.slice(0,3)
tweet_json['Month_name'] = tweet_json['created_at'].str.slice(4,7)
tweet_json['Month'] = tweet_json['Month_name'].map(calendar)
tweet_json['Day_number'] = tweet_json['created_at'].str.slice(8,10)
tweet_json['Tweeted_at_time'] = tweet_json['created_at'].str.slice(11,19)
tweet_json['Year'] = tweet_json['created_at'].str.slice(26,30)

In [ ]:
tweet_json['Data'] = str(str(tweet_json['Day_number']) + '-' + str(tweet_json['Month']) + '-' + str(tweet_json['Year']))

In [ ]:
tweet_json['Data'][0]

'0     15\n1     14\n2     14\n3     15\n4     14\n5     14\n6     15\n7     15\n8     14\n9     14\n10    15\n11    14\n12    14\n13    14\nName: Day_number, dtype: object-0     1\n1     1\n2     1\n3     1\n4     1\n5     1\n6     1\n7     1\n8     1\n9     1\n10    1\n11    1\n12    1\n13    1\nName: Month, dtype: int64-0     2022\n1     2022\n2     2022\n3     2022\n4     2022\n5     2022\n6     2022\n7     2022\n8     2022\n9     2022\n10    2022\n11    2022\n12    2022\n13    2022\nName: Year, dtype: object'

In [ ]:
# em média, quantos seguidores ganha após um tweet positivo? e quantos perde após um tweet negativo?
# modelo de classificação baseado na popularidade do participante
# prob de ser eliminado no paredão de acordo com o atual nível de popularidade